In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import glob
import pandas as pd
import tensorflow as tf
import cv2
from pathlib import Path
import time
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import resnet
from google.colab import drive
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras import backend, layers, metrics
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from google.colab.patches import cv2_imshow






In [ ]:
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/cv_data/*'
persons = glob.glob(path,)
persons

dictionary carry key -> name of person and value -> list of images that bleong to this person


x_train_real
    
     'personA' : [image1, image2, image3]
      'personB' : [image1, image2, image3]
      'perosnC' : [image1, image2, image3]


x_train_forged
  
    'personA' : [image1, image2, image3]
    'personB' : [image1, image2, image3]
    'perosnC' : [image1, image2, image3]

  

'''

In [ ]:
x_train_real = {}
x_train_forged= {}

x_test_real = {}
x_test_forged= {}

for person in persons:
  
  per = person.split('/cv_data/')[1]
  print( f'get Data of : {per}')
  x_train_real[per] = []
  x_train_forged[per] = []

  x_test_real[per] = []
  x_test_forged[per] = []
  
  train_image = glob.glob(f'{person}/Train/*.png')
  train_label = glob.glob(f'{person}/Train/*.csv')[0]
  df_train_label = pd.read_csv(train_label)


  test_image = glob.glob(f'{person}/Test/*.png')
  test_label = glob.glob(f'{person}/Test/*.csv')[0]
  df_test_label = pd.read_csv(test_label)
 
  ## read train image and divided into real and fake
  for image in train_image:
    name_of_image = image.split('/Train/')[1]
    img = cv2.imread(image,0)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  

    if df_train_label[df_train_label['image_name'] == name_of_image]['label'].count() == 1:
      lable = df_train_label[df_train_label['image_name'] == name_of_image].iloc[0]['label']
     
      if (lable == 'real'):
        x_train_real[per].append(img)
      else:
        x_train_forged[per].append(img)
  
  print(f'number of real Signature train : {len(x_train_real[per])}')
  print(f'number of forged Signature train : {len(x_train_forged[per])}')
  
## read test image and divided into real and fake
  for image in test_image:
    name_of_image = image.split('/Test/')[1]
    img = cv2.imread(image,0)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    if df_test_label[df_test_label['image_name'] == name_of_image]['label'].count() == 1:
      lable = df_test_label[df_test_label['image_name'] == name_of_image].iloc[0]['label']
     
      if (lable == 'real'):
        x_test_real[per].append(img)
      else:
        x_test_forged[per].append(img)

  print(f'number of real Signature test : {len(x_test_real[per])}')
  print(f'number of forged Signature test : {len(x_test_forged[per])}')

  
  print('======================================================================')

In [ ]:
def make_Triplets(real_image,forged_image):
  size_h = 128
  size_w =128
  tiplets =[]
 
 
  for key,images in real_image.items(): 

    lable_person = key

    for i in range(len(images)):
      for j in range(i+1,len(images)):

        AnchorImage = images[i]
        AnchorImage = cv2.resize(AnchorImage,(size_h,size_w))
      
        PostiveImage = images[j]
        PostiveImage =cv2.resize(PostiveImage,(size_h,size_w))
        
        NegativeImage =np.random.choice(x_train_forged[lable_person])
        NegativeImage = cv2.resize(NegativeImage,(size_h,size_w))
       
        tiplets.append((AnchorImage,PostiveImage,NegativeImage))

  return tiplets  

pairImage_train =  make_Triplets(x_train_real,x_train_forged)
pairImage_test =   make_Triplets(x_test_real,x_test_forged)

print(f'length of triple train : {len(pairImage_train)}')
print(f'length of triple train : {len(pairImage_test)}')


In [ ]:
def get_batch(triplet_list, batch_size=16, preprocess=True):
    batch_steps = len(triplet_list)//batch_size
    
    for i in range(batch_steps+1):
        anchor   = []
        positive = []
        negative = []
        
        j = i*batch_size
        while j<(i+1)*batch_size and j<len(triplet_list):
            a, p, n = triplet_list[j]
            anchor.append(a)
            positive.append(p)
            negative.append(n)
            j+=1
            
        anchor = np.array(anchor)
        positive = np.array(positive)
        negative = np.array(negative)
        
        if preprocess:
            anchor = preprocess_input(anchor)
            positive = preprocess_input(positive)
            negative = preprocess_input(negative)
        
        yield ([anchor, positive, negative])

In [ ]:
num_plots = 6

f, axes = plt.subplots(num_plots, 3, figsize=(15, 20))

for x in get_batch(pairImage_train, batch_size=num_plots, preprocess=False):
    a,p,n = x
    for i in range(num_plots):
        axes[i, 0].imshow(a[i])
        axes[i, 1].imshow(p[i])
        axes[i, 2].imshow(n[i])
        
        i+=1
    break

In [ ]:
def get_encoder(input_shape):
    
## pre - Trained Model Xception 
    pretrained_model = Xception(
        input_shape=input_shape,
        weights='imagenet',
        include_top=False,
        pooling='avg',
    )

## make all layers not train except last 27 layer of Xception
    for i in range(len(pretrained_model.layers)-27):
        pretrained_model.layers[i].trainable = False

## make layers after Xception that get from it emadding 
    encode_model = Sequential([
        pretrained_model,
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(128, activation="relu"),
        layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
    ], name="Encode_Model")
    
    return encode_model

In [ ]:
class DistanceLayer(layers.Layer):
    # A layer to compute ‖f(A) - f(P)‖² and ‖f(A) - f(N)‖²
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)

In [ ]:
def get_siamese_network(input_shape = (128, 128, 3)):
    encoder = get_encoder(input_shape)
    
    # Input Layers for the images
    anchor_input   = layers.Input(input_shape, name="Anchor_Input")
    positive_input = layers.Input(input_shape, name="Positive_Input")
    negative_input = layers.Input(input_shape, name="Negative_Input")
    
    ## Generate the encodings (feature vectors) for the images
    encoded_a = encoder(anchor_input)
    encoded_p = encoder(positive_input)
    encoded_n = encoder(negative_input)
    
    # A layer to compute ‖f(A) - f(P)‖² and ‖f(A) - f(N)‖²
    distances = DistanceLayer()(
       encoded_a,
       encoded_p,
       encoded_n,
    )
    
    # Creating the Model
    siamese_network = Model(
        inputs  = [anchor_input, positive_input, negative_input],
        outputs = distances,
        name = "Siamese_Network"
    )
    return siamese_network

siamese_network = get_siamese_network()
siamese_network.summary()

In [ ]:
plot_model(siamese_network, show_shapes=True, show_layer_names=True)

In [ ]:
class SiameseModel(Model):
    # Builds a Siamese model based on a base-model
    def __init__(self, siamese_network, margin=1.0):
        super(SiameseModel, self).__init__()
        
        self.margin = margin
        self.siamese_network = siamese_network
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape get the gradients when we compute loss, and uses them to update the weights
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)
            
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.siamese_network.trainable_weights))
        
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)
        
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # Get the two distances from the network, then compute the triplet loss
        ap_distance, an_distance = self.siamese_network(data)
        loss = tf.maximum(ap_distance - an_distance + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics so the reset_states() can be called automatically.
        return [self.loss_tracker]

In [ ]:
siamese_model = SiameseModel(siamese_network)
optimizer = Adam(learning_rate=1e-3, epsilon=1e-01)
siamese_model.compile(optimizer=optimizer)

In [ ]:
def test_on_triplets(batch_size = 256):
    pos_scores, neg_scores = [], []

    for data in get_batch(pairImage_test, batch_size=batch_size):
        prediction = siamese_model.predict(data)
        pos_scores += list(prediction[0])
        neg_scores += list(prediction[1])
    
    accuracy = np.sum(np.array(pos_scores) < np.array(neg_scores)) / len(pos_scores)

    ap_mean = np.mean(pos_scores)
    an_mean = np.mean(neg_scores)
    
    ap_stds = np.std(pos_scores)
    an_stds = np.std(neg_scores)
    
    print(f"Accuracy on test = {accuracy:.5f}")
    return (accuracy, ap_mean, an_mean, ap_stds, an_stds)

In [ ]:
save_all = False
epochs = 6
batch_size = 256

max_acc = 0
train_loss = []
test_metrics = []

for epoch in range(1, epochs+1):
    
    t = time.time()
    
    # Training the model on train data
    epoch_loss = []
    for data in get_batch(pairImage_train, batch_size=batch_size):
        loss = siamese_model.train_on_batch(data)
        epoch_loss.append(loss)
    epoch_loss = sum(epoch_loss)/len(epoch_loss)
    train_loss.append(epoch_loss)

    print(f"\nEPOCH: {epoch} \t (Epoch done in {int(time.time()-t)} sec)")
    print(f"Loss on train    = {epoch_loss:.5f}")
    
    # Testing the model on test data
    metric = test_on_triplets(batch_size=batch_size)
    test_metrics.append(metric)
    accuracy = metric[0]
    
    # Saving the model weights
    if save_all or accuracy>=max_acc:
        siamese_model.save_weights("siamese_model")
        max_acc = accuracy

# Saving the model after all epochs run
siamese_model.save_weights("siamese_model-final")

In [ ]:
def extract_encoder(model):
    encoder = get_encoder((128, 128, 3))
    i=0
    for e_layer in model.layers[0].layers[3].layers:
        layer_weight = e_layer.get_weights()
        encoder.layers[i].set_weights(layer_weight)
        i+=1
    return encoder

encoder = extract_encoder(siamese_model)
encoder.save_weights("encoder")
#encoder.save("/content/drive/MyDrive/cv_data/model.h5")
encoder.summary()

In [ ]:
def classify_images(Image1,Image2, threshold=1.3):
    # Getting the encodings for the passed faces
    embedding1 = encoder.predict(Image1)
    embedding2 = encoder.predict(Image2)
    
    distance = np.sum(np.square(embedding1-embedding2), axis=-1)
    prediction = np.where(distance<=threshold, 0, 1)
    return prediction

In [ ]:
def ModelMetrics(pos_list, neg_list):
    true = np.array([0]*len(pos_list)+[1]*len(neg_list))
    pred = np.append(pos_list, neg_list)
    
    # Compute and print the accuracy
    print(f"\nAccuracy of model: {accuracy_score(true, pred)*100}%\n")
    
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(true, pred)

    categories  = ['Similar','Different']
    names = ['True Similar','False Similar', 'False Different','True Different']
    percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]

    labels = [f'{v1}\n{v2}' for v1, v2 in zip(names, percentages)]
    labels = np.asarray(labels).reshape(2,2)

    sns.heatmap(cf_matrix, annot = labels, cmap = 'Blues',fmt = '',
                xticklabels = categories, yticklabels = categories)

    plt.xlabel("Predicted", fontdict = {'size':14}, labelpad = 10)
    plt.ylabel("Actual"   , fontdict = {'size':14}, labelpad = 10)
    plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)


pos_list = np.array([])
neg_list = np.array([])

for data in get_batch(pairImage_test, batch_size=len(pairImage_test)):
    print('a')
    a, p, n = data
    pos_list = np.append(pos_list, classify_images(a, p))
    neg_list = np.append(neg_list, classify_images(a, n))
    break

ModelMetrics(pos_list, neg_list)